In [59]:
import csv
from collections import Counter
import collections
import numpy as np
import operator
import h5py
import argparse
import sys
import re
import codecs
import string
import itertools
import gc
import copy
from nltk import pos_tag
from nltk.tag import UnigramTagger
import sys
import future
sys.path.append("/usr/local/lib/python2.7/site-packages")

import pandas as pd

from itertools import chain
from collections import Counter

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

In [60]:
FILE_PATHS = {"full": ("data/train.num.txt",
                      "data/dev.num.txt",
                      "data/dev_kaggle.txt",
                      "data/tags.txt",
                      "data/test.num.txt",
                      "data/glove.txt")}

train_path, valid_path, valid_kaggle_path, tags_path, test_path, embeddings_path = FILE_PATHS["full"]

train = (list(csv.reader(open(train_path, 'rb'), delimiter='\t', quotechar='|')))
valid = (list(csv.reader(open(valid_path, 'rb'), delimiter='\t',quotechar='|')))
valid_kaggle = (list(csv.reader(open(valid_kaggle_path, 'rb'), delimiter='\t', quotechar='|')))
test = (list(csv.reader(open(test_path, 'rb'), delimiter=' ', quotechar='|')))
tags = (list(csv.reader(open(tags_path, 'rb'), delimiter=' ')))

dataset = [train, valid, test]

def is_float(c):
    if c.isdigit():
        return True
    if c == ',':
        return True
    return False

def word_process(word):
    #Realized that this can be done much easier with regular expresions...
    #Removes digits and replaces them with the string 'NUMBER'
    flag = 0
    interval = [0,0]
    intervals = []
    processed = ''
    if len(word) == 1 and (not word.isdigit()):
        return word
    for i in range(0,len(word)):
        if flag == 0:
            if is_float(word[i]):
                flag = 1
                interval[0] = i
                #if len(word) == 1:
                    #intervals.append((0,0))
                if i == len(word)-1:
                    intervals.append((interval[0], interval[0]+1))
        else:
            if (not is_float(word[i])):
                flag = 0
                interval[1] = i-1
                intervals.append((interval[0],interval[1]))
            elif i == len(word) -1:
                interval[1] = i
                intervals.append((interval[0],interval[1]))
                
    index = 0
    for pair in intervals:
        prefix = word[index:pair[0]]
        index = pair[1]+1
        processed = processed + prefix + 'NUMBER'
    
    processed = processed + word[index:]
    return processed      

def gen_dictionary(data_set, tag_set):
    d = collections.defaultdict(list)
    d['<s>'] = 1
    d['</s>'] = 2
    counter = 3
    for data in data_set:
        for element in data:
            if element != []:
                word = word_process(element[2])
                if word not in d:
                    d[word] = counter
                    counter = counter + 1
    for i in range(0, len(tag_set)):
        tag_set[i][1] = int(tag_set[i][1])
    tag_set = dict(tag_set)
    tag_set['<s>'] = 8
    tag_set['</s>'] = 9
    
    return d, tag_set

def lex_features(data, v_dict):
    lex = []
    for i in range(0,len(data)):
        row = data[i]
        if row != []:
            if int(row[1]) == 1:
                lex.append(v_dict['<s>'])
            lex.append(v_dict[word_process(row[2])])
    lex = np.array(lex)
    lex = np.reshape(lex, (1, len(lex)))
    return np.array(lex)

def prev_features(data, t_dict):
    prev = [1]
    for i in range(0, len(data)-1):
        ind = data[i]
        prev.append(ind)
    return np.array(prev)


def get_outputs(data, t_dict):
    outputs = []
    for i in range(0, len(data)):
        row = data[i]
        if row != []:
            if int(row[1]) == 1:
                outputs.append(t_dict['<s>'])
            outputs.append(t_dict[row[3]])
    return np.array(outputs)

def get_features(data, v_dict, t_dict, feature_functions, outputs, memm):
    inputs = feature_functions[0](data, v_dict)
    outputs = np.insert(outputs, 0, outputs[0])[0:-1]
    if memm == True:
        inputs = np.vstack((inputs, outputs))
    for i in range(1, len(feature_functions)):
        inputs = np.vstack((inputs, feature_functions[0](data,v_dict)))
    return inputs.transpose()

def max_len(data):
    maxlen = 0
    count = 0
    for i in range(0, len(data)):
        row = data[i]
        if row != []:
            maxlen = max(int(row[1]), maxlen)
            if int(row[1]) == 1:
                count = count + 1
    return maxlen, count

def get_sentences(X_data, maxlen, numsen):
    #takes in the index form of the X data, maximum sentence length, and number of sentences
    sentences = np.zeros((numsen, maxlen+1, X_data.shape[1]))
    sen_ind = -1
    word_ind = 0
    for ind in X_data:
        if ind[0] == 1:
            sen_ind = sen_ind + 1
            word_ind = 0
        sentences[sen_ind, word_ind, :] = ind
        word_ind = word_ind + 1
    return sentences.astype(int)
         
features = [lex_features]

vocab_dict, tag_dict = gen_dictionary(dataset, tags)
Y_train = get_outputs(train, tag_dict)
Y_valid = get_outputs(valid, tag_dict)
X_train = get_features(train, vocab_dict, tag_dict, features, Y_train, False)
X_valid = get_features(valid, vocab_dict, tag_dict, features, Y_valid, False)
X_test = get_features(test, vocab_dict, tag_dict, features, Y_train, False)

#sentences
max_length_valid, num_sentence_valid = max_len(valid)
max_length_test, num_sentence_test = max_len(test)

#MEMM
X_train_MEMM = get_features(train, vocab_dict, tag_dict, features, Y_train, True)
X_valid_MEMM = get_features(valid, vocab_dict, tag_dict, features, Y_valid, True)
X_valid_sen_MEMM = get_sentences(X_valid_MEMM, max_length_valid, num_sentence_valid)



X_valid_sen = get_sentences(X_valid, max_length_valid, num_sentence_valid)
X_test_sen = get_sentences(X_test, max_length_test, num_sentence_test)

In [61]:
def sen_format(Y_train):
    tags = []
    sen = []
    for i in xrange(len(Y_train)):
        if i == 0:
            sen.append(Y_train[i])
        elif Y_train[i] == 8:
            tags.append(sen)
            sen = [8]
        else:
            sen.append(Y_train[i])
    return tags

In [62]:
def word2features(sent, i):
    word = sent[i][0]
    postag = pos_tag([word])[0][1]

    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = pos_tag([sent[i-1][0]])[0][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = pos_tag(sent[i+1][0])[0][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]    
 

In [63]:
def sen_format2(tags, test):
    new_train3 = []
    counter = 0
    for i in tags:
        temp = []
        for _ in xrange(len(i)):
            if len(test[counter]) >0:
                temp.append((test[counter][2],test[counter][3]))
            counter += 1
        new_train3.append(temp)
        temp = []
    return new_train3

In [64]:
train_tag = sen_format(Y_train)
valid_tag = sen_format(Y_valid)
to_train = sen_format2(train_tag, train)
to_valid = sen_format2(valid_tag, valid)


In [70]:
#train_sents = to_train
test_sents = to_valid

#X_train = [sent2features(s) for s in train_sents]
#y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [35]:
ttest = []
tempp = []
for i in test:
    if len(i) > 0:
        tempp.append((i[2], ))
    else:
        ttest.append(tempp)
        tempp = []

In [43]:
train_se

[('EU', 'I-ORG'),
 ('rejects', 'O'),
 ('German', 'I-MISC'),
 ('call', 'O'),
 ('to', 'O'),
 ('boycott', 'O'),
 ('British', 'I-MISC'),
 ('lamb', 'O'),
 ('.', 'O')]

In [53]:
kaggle_test = [sent2features(s) for s in ttest]

In [66]:
%%time
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)
 

CPU times: user 677 ms, sys: 17.7 ms, total: 694 ms
Wall time: 691 ms


In [57]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 1000,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})
 

In [67]:
trainer.train('cui-crf')
!ls -lh ./cui-crf


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 36457
Seconds required: 0.242

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 2147483647
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 35357.225174
Feature norm: 1.000000
Error norm: 14435.233486
Active features: 36457
Line search trials: 1
Line search step: 0.000009
Seconds required for this iteration: 0.101

***** Iteration #2 *****
Loss: 33318.434636
Feature norm: 1.081308
Error norm: 13135.359122
Active features: 36457
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.055

***** Iteration #3 *****
Loss: 21963.514711
Feature norm: 2.041568
Error norm: 6197.319321
Active features: 36457
Line search trials: 1
Line search step: 1.000000
Seconds requir

In [68]:
tagger = pycrfsuite.Tagger()
tagger.open('cui-crf')
 

In [69]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [73]:
%%time
y_pred = [tagger.tag(xseq) for xseq in kaggle_test]
 

CPU times: user 211 ms, sys: 7.25 ms, total: 218 ms
Wall time: 217 ms


In [72]:
print(bio_classification_report(Y_test, y_pred))

             precision    recall  f1-score   support

      I-LOC       0.78      0.76      0.77      1007
     I-MISC       0.86      0.75      0.80       511
      I-ORG       0.73      0.65      0.68       939
      I-PER       0.83      0.85      0.84      1355

avg / total       0.79      0.76      0.78      3812



In [32]:
Y_test = []
for i in test_sents:
    t = []
    for j in i:
        t.append(j[1])
    Y_test.append(t)
    t = []

In [74]:
with open("crf.csv", "w") as f:
    f.write("ID,Labels\n")
    for i in xrange(len(y_pred)):
        s = ""
        dic = {"I-PER":"PER", "O":"O", "I-LOC":"LOC","I-ORG":"ORG","I-MISC":"MISC","B-MISC":"MISC"}
        prev_tag = 0
        prev_idx = 0
        for j in xrange(len(y_pred[i])):
            if y_pred[i][j] != "O":
                if prev_tag == 0:
                    s += dic[y_pred[i][j]] + "-" + str(j+1)
                elif y_pred[i][j] == prev_tag and prev_idx == j-1:
                    s += "-" + str(j+1)
                else:
                    s += " " + dic[y_pred[i][j]] + "-" + str(j+1)
                prev_tag = y_pred[i][j]
                prev_idx = j
        f.write(str(i+1) + "," + s + "\n")
    f.close()
        